In [170]:
from langchain_community.document_loaders import PyPDFLoader
import os
import getpass
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
openai_api_key = os.getenv('OPENAI_API_KEY', 'YourAPIKeyIfNotSet')

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')



In [174]:
loader = PyPDFDirectoryLoader("/Users/kaitokikuchi/Library/CloudStorage/Dropbox/LLM-agriculture/PDFs")
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
docs = loader.load_and_split(text_splitter=splitter)

Illegal character in Name Object (b'/QEOWUY+HG\x91n\x89p\x8ap\xce\xdf\xaf\xcc\xdf\x91\xcc-WinCharSetFFFF-H')
Illegal character in Name Object (b'/QEOWUY+HG\x91n\x89p\x8ap\xce\xdf\xaf\xcc\xdf\x91\xcc')
Illegal character in Name Object (b'/CRDEAK+HG\x91n\x89p\x8ap\xce\xdf\xaf\xcc\xdf\x91\xcc-WinCharSetFFFF-H')
Illegal character in Name Object (b'/CRDEAK+HG\x91n\x89p\x8ap\xce\xdf\xaf\xcc\xdf\x91\xcc')


In [175]:
print(docs[0].page_content)
print("-----")

020406080100
本年 平年発
生
ほ
場
割
合
図１うどんこ病（葉）の発生状況甚（51％以上）
多（26～50％）
中（11～25％）
少（1～10％）発病葉率(%)
020406080100
本年 平年発
生
ほ
場
割
合
図２うどんこ病（果実）の発生状況甚（31％以上）
多（16～30％）
中（6～15％）
少（1～5％）発病果率((%)
020406080100
本年 平年発
生
ほ
場
割
合
図３灰色かび病の発生状況甚（31％以上）
多（16～30％）
中（6～15％）
少（1～5％）発病果率((%)イチゴの病害虫の発生状況（４ 月上中旬） 
   
うどんこ 病 
葉及び果実での発生ほ場割合は 平年並の 状況でした （図１、図２） 。蔓延すると防除が困難になるの
で、発生初期から防除を徹底してください。  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
灰色かび病  
 巡回調査において発生は確認されませんでした（図３）。 本病害は 過湿条件で発生が多くなりますの
で、換気 に心掛けてくださ い。また、薬剤防除だけでは十分な効果がありませんので、り病した果実や
茎葉、果梗はほ場外に持ち 出して処分してください。
-----


In [176]:


def make_embedder(model_type, save_path):
    if model_type == "sup-simcse-ja-base":
        model_name = "cl-nagoya/sup-simcse-ja-base"
        model_kwargs = {'device': 'cpu'}
        encode_kwargs = {'normalize_embeddings': False}
        embedding_model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
            )
    else:  # Assuming OpenAIEmbeddings
        embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
    
    fs = LocalFileStore(save_path)
    cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedding_model, fs)
    return cached_embedder

def save_embeddings_and_ids_with_faiss(all_documents, cached_embedder, save_path):
    db = FAISS.from_documents(all_documents, cached_embedder)
    db.save_local(save_path)



In [177]:
model_type = "sup-simcse-ja-base"  # "sup-simcse-ja-base" or "OpenAIEmbeddings"
cached_embedder_dir = "data/cached_embeddings"
save_path = os.path.join(cached_embedder_dir, model_type)

cached_embedder = make_embedder(model_type, save_path)
save_embeddings_and_ids_with_faiss(docs, cached_embedder, save_path)

Load from cached embedding

In [178]:
model_type = "sup-simcse-ja-base"  # "sup-simcse-ja-base" or "OpenAIEmbeddings"
cached_embedder_dir = "data/cached_embeddings"
save_path = os.path.join(cached_embedder_dir, model_type)

model_name = "cl-nagoya/sup-simcse-ja-base"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
            )
fs = LocalFileStore(save_path)
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedding_model, fs)
db = FAISS.load_local(save_path, cached_embedder)

In [179]:

def search(query: str):
    print(query)
    embedded_query = embedding_model.embed_query(query)
    top_10_similar_docs = db.similarity_search_by_vector(embedded_query, k=10)
    for doc in top_10_similar_docs:
        print(doc.metadata)
        print(doc.page_content)
        print("-----")

In [180]:
search("令和4年度の福島県でのハスモンヨトウ発生状況は？")

令和4年度の福島県でのハスモンヨトウ発生状況は？
{'source': '/Users/kaitokikuchi/Library/CloudStorage/Dropbox/LLM-agriculture/PDFs/令和５年度 病害虫防除情報.pdf', 'page': 0}
令和５年度 病害虫防除情報  
 
令 和 ５ 年 ８ 月２９日 
発表：福島県病害虫防除所  
 
 
 
 
１ 対象作物：野菜類・花き類  
２ 病害虫：ハスモンヨトウ  
３ 対象地域： 全域 
４ 発 生 量：やや多い  
 
【発生状況等 】 
(ア) 県内全域 に設置している ハスモンヨトウ のフェロモントラップへの誘殺数が 、８月中下
旬に平年を大きく上回りました（図１～５ ）。 
(イ) 仙台管区気象台が８月２４ 日に発表した１か月予報では、 気温が平年より高く、本種の
発生が助長され ると考えられます。  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 誘殺数が多くなっています。  
防除対策を徹底し、被害を防ぎましょう。
-----
{'source': '/Users/kaitokikuchi/Library/CloudStorage/Dropbox/LLM-agriculture/PDFs/令和４年度 病害虫防除情報.pdf', 'page': 0}
令和４年度 病害虫防除情報  
 
令 和 ４ 年 ９ 月 ２８日 
発表：福島県病害虫防除所  
 
ハスモンヨトウ の誘殺数が多くなっています。  
防除対策を徹底、被害 を防ぎましょう。  
 
１ 対象作物：野菜類・花き類  
２ 病害虫：ハスモンヨトウ  
３ 対象地域： 中通り・会津 
４ 発 生 量：やや多い  
 
【発生状況等 】 
(ア) 中通り及び会津に 設置している ハスモンヨトウ のフェロモントラップへの誘殺数が 、９
月上中旬に平年を大きく上回りました（図１～４）。  
(イ) ９月中旬に 中通り南部の露地キュウリで幼虫の寄生を確認しています。  
(ウ) 仙台管区気象台が９月２２日 に発表した１か月予報では、 気温が平年より高く、本種の
発生が助長され ると考えられます。
-----
{'source'

In [181]:

llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever()) #chain_type="stuff" occasionally results in token size limit error

In [182]:

qa.invoke("令和4年度の福島県でのハスモンヨトウ発生状況は？")

{'query': '令和4年度の福島県でのハスモンヨトウ発生状況は？',
 'result': ' 令和4年度の福島県でのハスモンヨトウ発生状況は、やや多いとされています。中通り及び会津に設置しているフェロモントラップへの誘殺数が、9月上中旬に平年を大きく上回り、中通り南部の露地キュウリで幼虫の寄生が確認されています。また、仙台管区気象台の1か月予報では気温が平年より高く、本種の発生が助長されると考えられています。'}

In [184]:
query = "例年の炭疽病の発生時期はいつ？"
qa.invoke(query)

{'query': '例年の炭疽病の発生時期はいつ？',
 'result': ' 高温時期に発生しやすい。気温28℃の比較的高温条件で多発する。定植後秋口にすぐ症状が見られるものは、潜在感染した親株から採苗～育苗期に感染したと考えられる。'}

In [185]:
query = "イチゴのうどんこ病の防ぎ方を具体的に教えてください"
qa.invoke(query)

{'query': 'イチゴのうどんこ病の防ぎ方を具体的に教えてください',
 'result': ' 発生初期から防除を徹底することが重要です。また、発病株や発病が疑われる株を速やかに抜き取り、適切に処分することも大切です。さらに、病害虫防除所のホームページに掲載されている簡易検定方法を参考にしてください。'}

In [183]:
query = "紫外線はうどんこ病を防げる？"
qa.invoke(query)

{'query': '紫外線はうどんこ病を防げる？',
 'result': ' はい、紫外線はイチゴうどんこ病の発生を抑制する効果があります。特に夜間3時間かつ15～30μw/㎠の処理で高い効果が得られることが示されています。'}

In [88]:
import tiktoken
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


encoding = tiktoken.get_encoding("cl100k_base")
len(encoding.encode("紫外線はうどんこ病を防げる？"))


21

# Using OpenAI Embeddings
Note: as of 02/04/2024 Tiktoken has not been updated to include the [new OpenAI embedders](https://openai.com/blog/new-embedding-models-and-api-updates) so `Warning: model not found. Using cl100k_base encoding.` will be raised ([PR link](https://github.com/openai/tiktoken/pull/247))

In [101]:
model_type = "OpenAIEmbeddings"  # "sup-simcse-ja-base" or "OpenAIEmbeddings"
cached_embedder_dir = "data/cached_embeddings"
save_path = os.path.join(cached_embedder_dir, model_type)

cached_embedder = make_embedder(model_type, save_path)
save_embeddings_and_ids_with_faiss(docs, cached_embedder, save_path)

In [102]:
model_type = "OpenAIEmbeddings"  # "sup-simcse-ja-base" or "OpenAIEmbeddings"
cached_embedder_dir = "data/cached_embeddings"
save_path = os.path.join(cached_embedder_dir, model_type)

embedding_model = embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
fs = LocalFileStore(save_path)
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedding_model, fs)
db = FAISS.load_local(save_path, cached_embedder)
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=db.as_retriever())

In [103]:
query = "令和3年の福島県でのハスモンヨトウ発生状況は？"
qa.invoke(query)

AssertionError: 